In [1]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    import pandas as pd
    import pandas_datareader.data as web
    import pickle 
    import requests
    import tensorflow as tf
    import os
    from collections import deque
    import random
    from datetime import date

In [2]:
name = 'Double Recurrent Q-learning agent'
image_path = 'D:/kenneth/agent/images/'
path = 'D:/kenneth/trading/'
run_date=open(path+'run_date.txt').read()
start=('2019-01-01')
end=('2019-12-31')
days=30

In [3]:
if not os.path.exists(image_path+name):
        os.makedirs(image_path+name) 
with open('D:/kenneth/trading/sp500/sp500tickers.txt','r') as f:
    tickers=[line.rstrip('\n') for line in f]

In [4]:
class Model:
    def __init__(self, input_size, output_size, layer_size, learning_rate, name):
        with tf.variable_scope(name):
            self.X = tf.placeholder(tf.float32, (None, None, input_size))
            self.Y = tf.placeholder(tf.float32, (None, output_size))
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                cell = tf.nn.rnn_cell.LSTMCell(layer_size, state_is_tuple = False)
                self.hidden_layer = tf.placeholder(tf.float32, (None, 2 * layer_size))
                self.rnn,self.last_state = tf.nn.dynamic_rnn(inputs=self.X,cell=cell,
                                                        dtype=tf.float32,
                                                            initial_state=self.hidden_layer)
                self.logits = tf.layers.dense(self.rnn[:,-1], output_size)
                self.cost = tf.reduce_sum(tf.square(self.Y - self.logits))
                self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
class Agent:

    LEARNING_RATE = 0.003
    BATCH_SIZE = 32
    LAYER_SIZE = 256
    OUTPUT_SIZE = 3
    EPSILON = 0.5
    DECAY_RATE = 0.005
    MIN_EPSILON = 0.1
    GAMMA = 0.99
    MEMORIES = deque()
    COPY = 1000
    T_COPY = 0
    MEMORY_SIZE = 300
    
    def __init__(self, state_size, window_size, trend, skip):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        tf.reset_default_graph()
        self.INITIAL_FEATURES = np.zeros((4, self.state_size))
        self.model = Model(self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE,
                           'real_model')
        self.model_negative = Model(self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE,
                                   'negative_model')
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        self.trainable = tf.trainable_variables()
    
    def _assign(self, from_name, to_name):
        from_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=from_name)
        to_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=to_name)
        for i in range(len(from_w)):
            assign_op = to_w[i].assign(from_w[i])
            self.sess.run(assign_op)

    def _memorize(self, state, action, reward, new_state, dead, rnn_state):
        self.MEMORIES.append((state, action, reward, new_state, dead, rnn_state))
        if len(self.MEMORIES) > self.MEMORY_SIZE:
            self.MEMORIES.popleft()

    def _select_action(self, state):
        if np.random.rand() < self.EPSILON:
            action = np.random.randint(self.OUTPUT_SIZE)
        else:
            action = self.get_predicted_action([state])
        return action

    def _construct_memories(self, replay):
        states = np.array([a[0] for a in replay])
        new_states = np.array([a[3] for a in replay])
        init_values = np.array([a[-1] for a in replay])
        Q = self.sess.run(self.model.logits, feed_dict={self.model.X:states, 
                                                   self.model.hidden_layer:init_values})
        Q_new = self.sess.run(self.model.logits, feed_dict={self.model.X:new_states, 
                                                       self.model.hidden_layer:init_values})
        Q_new_negative = self.sess.run(self.model_negative.logits, 
                                  feed_dict={self.model_negative.X:new_states, 
                                             self.model_negative.hidden_layer:init_values})
        replay_size = len(replay)
        X = np.empty((replay_size, 4, self.state_size))
        Y = np.empty((replay_size, self.OUTPUT_SIZE))
        INIT_VAL = np.empty((replay_size, 2 * self.LAYER_SIZE))
        for i in range(replay_size):
            state_r, action_r, reward_r, new_state_r, dead_r, rnn_memory = replay[i]
            target = Q[i]
            target[action_r] = reward_r
            if not dead_r:
                target[action_r] += self.GAMMA * Q_new_negative[i, np.argmax(Q_new[i])]
            X[i] = state_r
            Y[i] = target
            INIT_VAL[i] = rnn_memory
        return X, Y, INIT_VAL
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array(res)
    
    def buy(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        init_value = np.zeros((1, 2 * self.LAYER_SIZE))
        for k in range(self.INITIAL_FEATURES.shape[0]):
            self.INITIAL_FEATURES[k,:] = state
        for t in range(0, len(self.trend) - 1, self.skip):
            action, last_state = self.sess.run([self.model.logits,self.model.last_state],
                                                feed_dict={self.model.X:[self.INITIAL_FEATURES],
                                                            self.model.hidden_layer:init_value})
            action, init_value = np.argmax(action[0]), last_state
            next_state = self.get_state(t + 1)
            
            if action == 1 and initial_money >= self.trend[t]:
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
#                 print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
            
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((close[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
#                 print(
#                     'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
#                     % (t, close[t], invest, initial_money)
#                 )
            
            new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)
            self.INITIAL_FEATURES = new_state
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        self.sess.close()
        return states_buy, states_sell, total_gains, invest
            
    
    def train(self, iterations, checkpoint, initial_money):
        for i in range(iterations):
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            init_value = np.zeros((1, 2 * self.LAYER_SIZE))
            for k in range(self.INITIAL_FEATURES.shape[0]):
                self.INITIAL_FEATURES[k,:] = state
            for t in range(0, len(self.trend) - 1, self.skip):
                if (self.T_COPY + 1) % self.COPY == 0:
                    self._assign('real_model', 'negative_model')
                
                if np.random.rand() < self.EPSILON:
                    action = np.random.randint(self.OUTPUT_SIZE)
                else:
                    action, last_state = self.sess.run([self.model.logits,
                                                  self.model.last_state],
                                                  feed_dict={self.model.X:[self.INITIAL_FEATURES],
                                                             self.model.hidden_layer:init_value})
                    action, init_value = np.argmax(action[0]), last_state
                    
                next_state = self.get_state(t + 1)
                
                if action == 1 and starting_money >= self.trend[t]:
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                    
                invest = ((starting_money - initial_money) / initial_money)
                new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)
                
                self._memorize(self.INITIAL_FEATURES, action, invest, new_state, 
                               starting_money < initial_money, init_value[0])
                self.INITIAL_FEATURES = new_state
                batch_size = min(len(self.MEMORIES), self.BATCH_SIZE)
                replay = random.sample(self.MEMORIES, batch_size)
                X, Y, INIT_VAL = self._construct_memories(replay)
                
                cost, _ = self.sess.run([self.model.cost, self.model.optimizer], 
                                        feed_dict={self.model.X: X, self.model.Y:Y,
                                                  self.model.hidden_layer: INIT_VAL})
                self.T_COPY += 1
                self.EPSILON = self.MIN_EPSILON + (1.0 - self.MIN_EPSILON) * np.exp(-self.DECAY_RATE * i)
            if (i+1) % checkpoint == 0:
                pass
#                 print('epoch: %d, total rewards: %f.3, cost: %f, total money: %f'%(i + 1, total_profit, cost,
#                                                                                   starting_money))

In [5]:
def plot_all(ticker):
    close = df['Close']
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=10, color='g', label = 'buying signal', markevery = states_buy)
    plt.plot(close, 'v', markersize=10, color='r', label = 'selling signal', markevery = states_sell)
    plt.title(name+'\n'+'Ticker: '+ticker)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'.png', facecolor='yellowgreen', dpi=100)
#     plt.show()
    plt.close()

In [6]:
def display_only(days):
    new_buy = []
    new_sell = []
    display_days = days
    total_days= df.shape[0]
    display_from = total_days-display_days
    for i in(states_buy):
        if i > display_from:
            adj = i- display_from
            new_buy.append(adj)
    for i in(states_sell):
        if i > display_from:
            adj = i- display_from
            new_sell.append(adj)
    
    return(new_buy, new_sell)

In [7]:
def plot_by_days(days, ticker):
    close = df['Close'][-days:]
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=12, color='g',alpha=1.0,label = 'buying signal', markevery = new_buy)
    plt.plot(close, 'v', markersize=12, color='r',alpha=1.0, label = 'selling signal', markevery = new_sell)
    plt.title(name+'\n'+'Last '+str(days)+' Daily Trade Recommendations')
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'_'+str(days)+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [8]:
def save_signals():
    path2 = image_path+name+'/signal/'
    if not os.path.exists(path2):
        os.makedirs(path2)
        
    if  (len(states_buy)!= 0) and (len(states_sell) != 0):  
        if int(states_buy[-1])>int(states_sell[-1]):
            last_signal = 'Trading Buy'
            signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
        else:
            last_signal = 'Trading Sell'
            signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
        
    elif len(states_buy)== 0 and len(states_sell)>0:
        last_signal = 'Trading Sell'
        signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)>0:
        last_signal = 'Trading Buy'
        signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)==0:
        last_signal = 'None'
        signal_day = 'None'  
        
    with open(path2+ticker+'.txt','w') as f:
        print(
            'model_run,','signal_day,','last_signal,','total_gains,','invest,','data_start','\n',
              str(date.today()),',',signal_day,',',last_signal,',',total_gains,',',invest,',',str(start),
              file=f)

In [9]:
initial_money = 10000
window_size = 30
skip = 1
iterations = 10
checkpoint = 10
batch_size = 32

for ticker in tickers[455:]:
    df = pd.read_csv(path+'sp500/'+run_date+'/'+ticker+'.csv', index_col=0, parse_dates=True)
    df =df[start:end]
    print('Started ticker:', ticker)
    
    close = df.Close.values.tolist()
   
    agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              skip = skip)
    agent.train(iterations = iterations, checkpoint = checkpoint, initial_money = initial_money)

    states_buy, states_sell, total_gains, invest = agent.buy(initial_money = initial_money)
    save_signals()
    plot_all(ticker)
#     new_buy, new_sell = display_only(days)
#     plot_by_days(days, ticker)


Started ticker: UAA
Started ticker: UAL
Started ticker: UDR
Started ticker: UHS
Started ticker: ULTA
Started ticker: UNH
Started ticker: UNM
Started ticker: UNP
Started ticker: UPS
Started ticker: URI
Started ticker: USB
Started ticker: UTX
Started ticker: V
Started ticker: VAR
Started ticker: VFC
Started ticker: VIAB
Started ticker: VLO
Started ticker: VMC
Started ticker: VNO
Started ticker: VRSK
Started ticker: VRSN


Started ticker: VRTX
Started ticker: VTR
Started ticker: VZ
Started ticker: WAB
Started ticker: WAT
Started ticker: WBA
Started ticker: WCG
Started ticker: WDC
Started ticker: WEC
Started ticker: WELL
Started ticker: WFC
Started ticker: WHR
Started ticker: WLTW
Started ticker: WM
Started ticker: WMB
Started ticker: WMT
Started ticker: WRK
Started ticker: WU
Started ticker: WY
Started ticker: WYNN
Started ticker: XEC


Started ticker: XEL
Started ticker: XLB
Started ticker: XLE
Started ticker: XLF
Started ticker: XLI
Started ticker: XLK
Started ticker: XLNX
Started ticker: XLP
Started ticker: XLU
Started ticker: XLV
Started ticker: XLY
Started ticker: XOM
Started ticker: XRAY
Started ticker: XRT
Started ticker: XRX
Started ticker: XYL
Started ticker: YUM
Started ticker: ZBH
Started ticker: ZION
Started ticker: ZTS
Started ticker: ^GSPC


Started ticker: ^IXIC
Started ticker: ^NDX
Started ticker: ^SOX
Started ticker: ^TNX
Started ticker: ^VIX
Started ticker: ^VVIX
Started ticker: ^VXN
Started ticker: ^VXO
Started ticker: ^VXV
